In [1]:
from google.cloud import bigquery

In [37]:
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
(
   SELECT RAND() AS splitfield,
   arrival_delay,
   departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [38]:
results = bigquery.Client().query(compute_alpha).to_dataframe()

In [39]:
results

,alpha
0,0.97692


In [40]:
alpha = results["alpha"][0]
print(alpha)

0.9769197757932846


In [42]:
results2 = bigquery.Client().query(compute_alpha).to_dataframe()
alpha2 = results2["alpha"][0]
print(alpha2)

0.9767533411410161


In [9]:
compute_rmse = """
#standardSQL
SELECT
  dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay)*(arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM (
  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX' )
GROUP BY
  dataset
"""
result1 = bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe()
result1

,dataset,rmse,num_flights
0,train,13.066266,63681
1,eval,13.153360,16008


In [11]:
result2 = bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe()
result2

,dataset,rmse,num_flights
0,train,13.078286,63755
1,eval,13.105881,15934


In [14]:
train_and_eval_rand = """
    WITH
        alldata as (
            select if( rand() < 0.8, 'train', 'eval') as dataset,
                   arrival_delay,
                   departure_delay
            from `bigquery-samples.airline_ontime_data.flights`
            where departure_airport = 'DEN'
                  and arrival_airport = 'LAX'
        ),
        training as (
            select SAFE_DIVIDE( SUM(arrival_delay * departure_delay) , SUM(departure_delay * departure_delay)) AS alpha
            from alldata
            where dataset = 'train'
        )
    select MAX(alpha) AS alpha,
           dataset,
           SQRT(AVG((arrival_delay - alpha * departure_delay)*(arrival_delay - alpha * departure_delay))) AS rmse,
           COUNT(arrival_delay) AS num_flights
    from alldata, training
    group by dataset
"""

In [15]:
bigquery.Client().query(train_and_eval_rand).to_dataframe()

,alpha,dataset,rmse,num_flights
0,0.973376,eval,13.144531,16025
1,0.973376,train,13.068698,63664


In [16]:
bigquery.Client().query(train_and_eval_rand).to_dataframe()

,alpha,dataset,rmse,num_flights
0,0.975557,train,13.030102,63756
1,0.975557,eval,13.296405,15933


# Using HASH of date to split the data

In [44]:
compute_alpha = """
    #standardSQL
    select safe_divide( sum(arrival_delay * departure_delay),sum(departure_delay * departure_delay) ) as alpha
    from `bigquery-samples.airline_ontime_data.flights`
    where departure_airport = 'DEN' 
          and arrival_airport = 'LAX'
          and MOD( ABS(FARM_FINGERPRINT(date)),10 ) < 8
"""

In [45]:
results = bigquery.Client().query(compute_alpha).to_dataframe()

In [46]:
alpha = results['alpha'][0]
print(alpha)

0.9758039143620403


In [47]:
alpha = results['alpha'][0]
print(alpha)

0.9758039143620403


In [48]:
compute_rmse = """
    #standardSQL
    select if( mod(abs(FARM_FINGERPRINT(date)),10) < 8, 'train', 'eval') as dataset,
           sqrt( avg( (arrival_delay - ALPHA * departure_delay) * (arrival_delay - ALPHA * departure_delay) ) ) AS rmse,
           count(arrival_delay) as num_fights
    from `bigquery-samples.airline_ontime_data.flights`
    where departure_airport = 'DEN'
          and arrival_airport = 'LAX'
    group by dataset
"""

In [49]:
print(bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe().head())

  dataset       rmse  num_fights
0    eval  12.764685       15671
1   train  13.160712       64018


In [50]:
print(bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe().head())

  dataset       rmse  num_fights
0    eval  12.764685       15671
1   train  13.160712       64018
